In [61]:
%load_ext autoreload
%autoreload 2
import param
import panel as pn
import holoviews as hv
import numpy as np
from bokeh.models.formatters import PrintfTickFormatter, FuncTickFormatter
from sympy import lambdify

from symbolic_solver import analytic_solution, analytic_function
pn.extension()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
class AnalyticSol(param.Parameterized):
    distance = param.Number(default = 100)
    radius = param.Number(default = 16)
    latitude = param.Number(default = -20)
    speed = param.Number(default = -2)
    drag = param.Number(default = -3)
    slope = param.Number(default = -2)
    
    _,_,_,sol = analytic_solution()
    
    
    @param.depends('distance', 'radius','latitude','speed','drag','slope')
    def substitute(self):
        
        svals = np.linspace(0,self.distance*1e3,1000) 
        coriolis = 2*7.29e-5*np.sin(self.latitude*np.pi/180)
        anafun = analytic_function(self.sol, self.radius*1e3, coriolis, 10**self.speed, 10**self.slope, 10**self.drag)
        
        rvals =  anafun(svals).real/1e3
        coords = np.array( [svals/1e3, rvals]).T
        
        ymin = max(rvals.min(),0)
        return hv.Curve(coords).opts(color = "k", framewise = True, title = f"Ro = {10**self.speed/(self.radius*1e3*coriolis):.3f}",
                                    ylim = (ymin,None), xlim = (0,None), xlabel = "s (km)", ylabel = "r (km)")
    
    def view(self):
        widgets = {}
        widgets["distance"] = pn.Param(self.param.distance, widgets = {"distance": pn.widgets.FloatSlider(name = "Integration Distance", start = 1, end = 100, step = 1, value = 50, format=PrintfTickFormatter(format='%d km'))} )
        widgets["radius"] = pn.Param(self.param.radius, widgets = {"radius": pn.widgets.FloatSlider(name = "Radius (km)", start = 1, end = 50, step = 0.5, value = 16, format=PrintfTickFormatter(format='%d km'))} )
        widgets["latitude"] = pn.Param(self.param.latitude, widgets = {"latitude": pn.widgets.FloatSlider(name = "Latitude", start = -90, end = 90, step = 1, value = -20, format=PrintfTickFormatter(format='%.1f°'))} )
        widgets["speed"] = pn.Param(self.param.speed, widgets = {"speed": pn.widgets.FloatSlider(name = "Speed (m/s) ", start = -3, end = 0, step = 0.025, value = -2, format=FuncTickFormatter(code="""return (10**tick).toPrecision(3)"""))} )
        widgets["slope"] = pn.Param(self.param.slope, widgets = {"slope": pn.widgets.FloatSlider(name = "Slope (𝛽)", start = -3, end = 0, step = .05, value = -2, format=FuncTickFormatter(code="""return (10**tick).toPrecision(3)"""))} )
        widgets["drag"] = pn.Param(self.param.drag, widgets = {"drag": pn.widgets.FloatSlider(name = "Quadratic drag coefficient (Cd)", start = -3, end = 0, step = .05, value = -3, format=FuncTickFormatter(code="""return (10**tick).toPrecision(3)"""))} )
       
        return pn.Row( pn.WidgetBox('### IVP Parameters', widgets["distance"], widgets["radius"], widgets["latitude"], widgets["speed"], widgets["slope"], widgets["drag"], width = 400) 
                      ,hv.DynamicMap(self.substitute).opts(data_aspect = 1, frame_height = 500, frame_width = 1000, fontsize={'title': 16, 'labels': 16, 'ticks': 12}) )

A = AnalyticSol()
A.view()

Row
    [0] WidgetBox(width=400)
        [0] Markdown(str)
        [1] Param(AnalyticSol, parameters=['distance'], show_name=False, widgets={'distance': FloatSlider(e...})
        [2] Param(AnalyticSol, parameters=['radius'], show_name=False, widgets={'radius': FloatSlider(end...})
        [3] Param(AnalyticSol, parameters=['latitude'], show_name=False, widgets={'latitude': FloatSlider(e...})
        [4] Param(AnalyticSol, parameters=['speed'], show_name=False, widgets={'speed': FloatSlider(end=...})
        [5] Param(AnalyticSol, parameters=['slope'], show_name=False, widgets={'slope': FloatSlider(end=...})
        [6] Param(AnalyticSol, parameters=['drag'], show_name=False, widgets={'drag': FloatSlider(end=0...})
    [1] HoloViews(DynamicMap)